In [1]:
from keras.datasets import fashion_mnist
from art.utils import load_mnist
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Lambda, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential,load_model
from art.metrics.metrics import empirical_robustness, clever_t, clever_u, clever, loss_sensitivity, wasserstein_distance
import numpy as np
from art.estimators.classification.keras import KerasClassifier
import logging
from progressbar import ProgressBar

import sys
sys.path.append('..')  
from GradPri_utils.utils import *

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.compat.v1.disable_eager_execution()

In [2]:
R_L1 = 40
R_L2 = 2
R_LI = 0.1

def _create_krclassifier_FMNIST():
    num_classes = 10
    input_shape = (28,28,1)
    
    model = Sequential()
    # Block 1
    model.add(Conv2D(64, (3, 3), padding='same', name = 'block1_conv1', 
                     input_shape=input_shape))
    # model.add(BatchNormalization(name = 'block1_bn1'))
    model.add(Activation('relu', name = 'block1_act1'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool1'))

    model.add(Conv2D(128, (3, 3), padding='same', name = 'block1_conv2'))
    # model.add(BatchNormalization(name = 'block1_bn2'))
    model.add(Activation('relu', name = 'block1_act2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool2'))

    model.add(Conv2D(256, (3, 3), padding='same', name = 'block1_conv3'))
    # model.add(BatchNormalization(name = 'block1_bn3'))
    model.add(Activation('relu', name = 'block1_act3'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool3'))
    # top
    model.add(Flatten(name='flatten'))
    model.add(Dense(1024, activation='relu', name='fc1'))
    model.add(Dropout(0.5, name='drop1'))
    model.add(Dense(32, activation='relu', name='fc2'))
    model.add(Dropout(0.5, name='drop2'))
    # model.add(Dense(64, activation='relu', name='fc3'))
    # model.add(Dropout(0.5, name='drop3'))
    model.add(Dense(num_classes, activation='softmax', name='predictions'))

    
    # model = load_model("./fmnist_CNN.h5")
    opt = tf.keras.optimizers.Adam(learning_rate=0.00005)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    # Get the classifier
    krc = KerasClassifier(model=model, clip_values=(0, 1), use_logits=False)

    return krc

logger = logging.getLogger(__name__)

In [6]:
# Get Fashion-MNIST
(x_train, y_train), (x_test, y_test), _, _ = load_mnist()

# Get the classifier
krc = _create_krclassifier_FMNIST()
krc.fit(x_train, y_train, batch_size=256, nb_epochs=5, shuffle=True,
        validation_data=(x_test[0:5000], y_test[0:5000]), verbose=1)

# checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="./fmnist_CNN.h5", 
#                                                     monitor='val_accuracy', 
#                                                     verbose=1, 
#                                                     save_best_only=True, mode = 'max')
# callbacks_list = [checkpoint]
# krc.fit(x_train, y_train, batch_size=256, epochs=20, shuffle=True,
#           validation_data=(x_test[0:5000], y_test[0:5000]), callbacks=callbacks_list)

Train on 60000 samples, validate on 5000 samples
Epoch 1/5
60000/60000 [==============================] - ETA: 0s - loss: 1.4646 - accuracy: 0.5161

/public/liujiawei/anaconda3/envs/ZHB_env/lib/python3.6/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


60000/60000 [==============================] - 6s 97us/sample - loss: 1.4646 - accuracy: 0.5161 - val_loss: 0.5114 - val_accuracy: 0.8656
Epoch 2/5
60000/60000 [==============================] - 2s 29us/sample - loss: 0.5943 - accuracy: 0.8117 - val_loss: 0.2430 - val_accuracy: 0.9350
Epoch 3/5
60000/60000 [==============================] - 2s 29us/sample - loss: 0.4002 - accuracy: 0.8759 - val_loss: 0.1657 - val_accuracy: 0.9494
Epoch 4/5
60000/60000 [==============================] - 2s 29us/sample - loss: 0.3208 - accuracy: 0.9035 - val_loss: 0.1333 - val_accuracy: 0.9588
Epoch 5/5
60000/60000 [==============================] - 2s 29us/sample - loss: 0.2718 - accuracy: 0.9187 - val_loss: 0.1123 - val_accuracy: 0.9644


In [7]:
Gini = []
total_sample_num = 2000
predicted_confidence = []

pbar = ProgressBar()
for index in pbar(range(total_sample_num)):
    pre_tmp = krc.predict(np.reshape(x_test[index], [-1,28,28,1]))
    Gini.append(1-np.sum(pre_tmp*pre_tmp))
    predicted_confidence.append(pre_tmp)

100% |########################################################################|


In [8]:
indexs = np.argsort(Gini)
indexs = indexs[::-1]
# 计算APFD指标
APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=y_test[range(total_sample_num)],
                   predicted_confidence=np.array(predicted_confidence))
print("APFD: ", APFD)

# 计算RAUC指标
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=y_test[range(total_sample_num)],
                    predicted_confidence=np.array(predicted_confidence))
print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.9396470588235294
RAUC:  0.9554000089738413


In [9]:
BoundaryGini = []

pbar = ProgressBar()
for index in pbar(range(total_sample_num)):
    pred = krc.predict(np.reshape(x_test[index], [-1,28,28,1]))
    res_tmp = clever_u(krc, x_test[index], 10, 10, R_LI, norm=np.inf, pool_factor=3, verbose=False)
    BoundaryGini.append(res_tmp)

100% |########################################################################|


In [10]:
indexs = np.argsort(BoundaryGini)
# indexs = indexs[::-1]
# 计算APFD指标
APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=y_test[range(total_sample_num)],
                   predicted_confidence=np.array(predicted_confidence))
print("APFD: ", APFD)

# 计算RAUC指标
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=y_test[range(total_sample_num)],
                    predicted_confidence=np.array(predicted_confidence))
print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.8601102941176471
RAUC:  0.8745083082813598
